In [ ]:
#install.packages('plotly')
library(knitr)
library(foreign)
library(ggplot2)
library(tidyverse)
library(rworldmap)
library(sp)
library(gganimate)
library(maps)
library(ggthemes)
library(plotly)
library(gifski)
library(gganimate)

In [ ]:
mapdata <- as.data.frame(read.csv('african_crises.csv'))
counts <- mapdata
counts$period[counts$year <= 1880] <- 1880
counts$period[1880 < counts$year & counts$year <= 1900] <- 1900
counts$period[1900 < counts$year & counts$year <= 1920] <- 1920
counts$period[1920 < counts$year & counts$year <= 1940] <- 1940
counts$period[1940 < counts$year & counts$year <= 1960] <- 1960
counts$period[1960 < counts$year & counts$year <= 1980] <- 1980
counts$period[1980 < counts$year & counts$year <= 2000] <- 2000
counts$period[2000 < counts$year] <- 2014
counts$banking_crises = ifelse(counts$banking_crisis == 'crisis', 1, 0)
counts$region = counts$country
counts = subset(counts, select = -c(country, banking_crisis, cc3))

grouped <- counts %>%
  group_by(region) %>%
    mutate(total_crises = 
           sum(systemic_crisis) + sum(inflation_crises) + sum(currency_crises) + sum(banking_crises)) %>%
    mutate(systemic_crisis = sum(systemic_crisis)) %>%
    mutate(inflation_crises = sum(inflation_crises)) %>%
    mutate(currency_crises = sum(currency_crises)) %>%
    mutate(banking_crises = sum(banking_crises)) 
grouped <- as.data.frame(grouped)
head(grouped)
#write.csv(grouped, 'crises_merged.csv', row.names = TRUE)

In [ ]:
grouped_sum <- counts %>%
  group_by(region) %>%
    mutate(total_crises = cumsum(systemic_crisis + banking_crises + inflation_crises + currency_crises))

grouped_sum <- as.data.frame(grouped_sum)

period_grouped <- grouped_sum[, c('period', 'region', 'total_crises')] %>%
  group_by(region, period) %>%
    mutate(total = max(total_crises))

period_grouped_merge <- unique(as.data.frame(period_grouped[, c('period', 'region', 'total')]))


In [ ]:
countries <- c('Algeria', 'Angola', 'Central African Republic', 'Ivory Coast',
    'Egypt', 'Kenya', 'Mauritius', 'Morocco', 'Nigeria', 'South Africa', 'Tunisia', 'Zambia', 'Zimbabwe')
periods <- c(1880, 1900, 1920, 1940, 1960, 1980, 2000, 2014)

merge_df <- data.frame("region" = character(0), "period" = numeric(0))

for (i in countries) {
    for (j in periods) {
        merge_df <- rbind(merge_df, data.frame(region=i, period=j))
    }
}

new_merge <- left_join(merge_df, period_grouped_merge, by=c('region', 'period'))
#VALUES MANUALLY FILLED IN 

In [ ]:
world_data <- map_data("world")
world_joined <- inner_join(world_data, grouped, by='region')
world_joined[is.na(world_joined)] <- ""

In [ ]:
world <- ggplot() +
  borders("world", colour = "gray85", fill = "gray80") + coord_fixed(1.1) +
    xlim(-20, 60) + ylim(-40, 40) + theme_map()

inter_map <- world +
  geom_polygon(aes(x = long, y = lat, text = paste('Country:', region, 
                                                  '<br /> Total crises:', total_crises, 
                                                  '<br /> Systemic crises:', systemic_crisis,
                                                  '<br /> Currency crises:', currency_crises,
                                                  '<br /> Inflation crises:', inflation_crises,
                                                  '<br /> Banking crises:', banking_crises),
                 group = region, fill=total_crises),
             data = world_joined, alpha = .5) +
  labs(fill = 'Number of total crises') + 
  theme(legend.position = c(0.1, 0.2))

inter_plot <- ggplotly(inter_map, tooltip = 'text')

In [ ]:
inter_plot

In [ ]:
#install.packages('gifski')
#devtools::install_github('thomasp85/transformr', type='binary')
time_data <- as.data.frame(read.csv('periods_merged.csv'))
world_joined2 <- inner_join(world_data, time_data, by='region')
world_joined2[is.na(world_joined2)] <- ""

growth_map <- world + 
    geom_polygon(aes(x = long, y = lat,
                 group = region, fill=total),
             data = subset(world_joined2, subregion != 'Marion Island'), alpha = 0.5) + 
    labs(title = "Number of crises experienced in {frame_time}") + 
    transition_time(period) + 
    labs(fill = 'Number of crises') + 
    theme(legend.position = c(0.1, 0.2))

anim <- animate(
    growth_map, duration = 5, fps = 20, width = 500, height = 500,
    nframes=50, renderer = gifski_renderer(), end_pause = 20)
anim_save(filename = 'time_map.gif', animation = anim)


In [ ]:
grouped_avg <- counts[, c('exch_usd', 'gdp_weighted_default', 
                           'inflation_annual_cpi', 'period', 'region')] %>%
  group_by(region, period) %>%
    mutate(average_exch = mean(exch_usd)) %>%
    mutate(average_gdp = mean(gdp_weighted_default)) %>%
    mutate(average_cpi = mean(inflation_annual_cpi))

grouped_avg <- unique(as.data.frame(grouped_avg[, 4:8]))

In [ ]:
group_combined <- left_join(time_data[, 2:4], grouped_avg, by=c('region', 'period'))
#write.csv(group_combined, 'periods_merged.csv', row.names=TRUE)